In [3]:
import os

In [6]:
pwd

'c:\\Users\\user\\Desktop\\End-to end- ML-project-with-ML-flow\\End-to-end-Machine-Learning-Project-with-Mlflow'

In [7]:
from  dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [8]:
from src.Mlproject.constants import *
from  src.Mlproject.utils.common import read_yaml, create_directories

In [9]:
import logging
from pathlib import Path
from box import ConfigBox

logging.basicConfig(level=logging.INFO, format='[%(asctime)s]: %(message)s:')

CONFIG_FILE_PATH = Path("config/config.yaml")
PARAMS_FILE_PATH = Path("params.yaml")
SCHEMA_FILE_PATH = Path("schema.yaml")

In [11]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
    ):
        self.config = ConfigBox(read_yaml(config_filepath))
        self.params = ConfigBox(read_yaml(params_filepath))
        self.schema = ConfigBox(read_yaml(schema_filepath))

        logging.info(f"Config contents: {self.config}")
        create_directories([self.config.artifacts_root])

   
    def get_model_trainer_config(self) -> ModelTrainerConfig:  
        config = self.config.model_trainer
        params = self.params.ElasticNet   
        schema = self.schema.TARGET_COLUMN    

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            alpha=params.alpha,
            l1_ratio = params.l1_ratio,
            target_column = schema.name
        )

        return model_trainer_config

In [12]:
import pandas as pd
import os
from Mlproject import logger 
from sklearn.linear_model import ElasticNet
import joblib

In [13]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)    
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis =1 )
        test_x = test_data.drop([self.config.target_column], axis =1)
        train_y = train_data[self.config.target_column]
        test_y = test_data[self.config.target_column]


        lr = ElasticNet(alpha=self.config.alpha, l1_ratio =self.config.l1_ratio, random_state=42 )
        lr.fit(train_x, train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))

In [14]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()  
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2025-01-27 06:56:50,613: INFO: common: YAML file: config\config.yaml loaded successfully]
[2025-01-27 06:56:50,944: INFO: common: YAML file: params.yaml loaded successfully]
[2025-01-27 06:56:50,971: INFO: common: YAML file: schema.yaml loaded successfully]
[2025-01-27 06:56:50,977: INFO: 505016316: Config contents: {'artifacts_root': 'artifacts', 'data_ingestion': {'local_data_file': 'artifacts/data_ingestion/data.zip', 'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/WisdomAnalyst/FIFA-21-DATA-CLEANING-/raw/refs/heads/main/winequality-data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'data_transformation': {'data_path': 'artifacts/data_ingestion/winequality-red.csv', 'root_dir': 'artifacts/data_transformation', 'test_data_path': 'artifacts/data_transformation/test.csv', 'train_data_path': 'artifacts/data_transformation/train.csv'}, 'data_validation': {'STATUS_FILE': 'artifacts/data_validation/status.txt', 'root_dir': 'artifacts/data_validation', 'unzip_da